# Playoff Fantasy Football 2021

#### Improvements



1) Make it so DFs of games auto-update with scores (started in get_scoring_data -- need to continue in scoring_by_player, etc) -- ie. add a column for the score in the game

2) Add a DF that tracks the perfect team to date

3) As I do with the Leaderboard DF, make it so that the DFs of games and scoring players from each weekend is on the same sheet



## Imports

In [1]:
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import re
import math
plt.style.use('fivethirtyeight')
import requests
from lxml import html
from lxml import etree

from bs4 import BeautifulSoup

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import openpyxl

from collections import Counter

## Create Teams

In [2]:
#SAVING TEAMS HERE
caseySquad = ["Josh Allen", "Tom Brady", 
              "Alvin Kamara", "Stefon Diggs", "Zach Moss", "Mike Evans", "Chris Godwin", 
              "Rob Gronkowski", "Ronald Jones", 
              "Tyler Bass", "Ryan Succop", "TB", "NO"]
kellySquad = ["Patrick Mahomes", "Tom Brady",
              "Alvin Kamara", "Davante Adams", "Aaron Jones", "Robert Tonyan", 
              "Travis Kelce", "Tyreek Hill", "Derrick Henry", 
              "Tyler Bass", "Harrison Butker", "GB", "TEN"]
johnMSquad = ["Josh Allen", "Tom Brady", 
              "Alvin Kamara", "Stefon Diggs", "Gabriel Davis", "Mike Evans", 
              "Ronald Jones","Antonio Brown", "Tyreek Hill", 
              "Tyler Bass", "Ryan Succop", "TB", "BUF"]
jamesSquad = ["Josh Allen", "Patrick Mahomes", 
              "Alvin Kamara", "Derrick Henry", "AJ Brown", "Davante Adams", 
              "Tyreek Hill", "Travis Kelce", "Tyler Lockett",
              "Harrison Butker", "Mason Crosby", "BUF", "NO"]
micahSquad = ["Patrick Mahomes", "Aaron Rodgers",
              "Alvin Kamara", "Davante Adams", "Tyreek Hill", "Travis Kelce", 
              "Derrick Henry", "Stefon Diggs", "DK Metcalf",
              "Mason Crosby", "Harrison Butker", "KC", "GB"]
timSquad = ["Josh Allen", "Aaron Rodgers", 
            "Alvin Kamara", "Stefon Diggs", "Davante Adams", "Aaron Jones",
            "Mike Evans", "Tyreek Hill", "Travis Kelce",
            "Tyler Bass", "Wil Lutz", "BUF", "NO"]
weezSquad = ["Patrick Mahomes", "Josh Allen",
             "Alvin Kamara", "Davante Adams", "Aaron Jones", "DK Metcalf",
             "Chris Carson", "Tyreek Hill", "Clyde Edwards-Helaire",
             "Jason Myers", "Harrison Butker", "BUF", "NO"]
skipSquad = ["Patrick Mahomes", "Tom Brady", 
             "Tyreek Hill", "Clyde Edwards-Helaire", "Chris Godwin", "Rob Gronkowski",
             "Derrick Henry", "DK Metcalf", "Chris Carson",
             "Harrison Butker", "Wil Lutz", "PIT", "TB"]
nickySquad = ["Patrick Mahomes", "Aaron Rodgers", 
              "Alvin Kamara", "Taysom Hill", "Davante Adams", "Aaron Jones", 
              "Tyreek Hill", "Travis Kelce", "Stefon Diggs",
              "Harrison Butker", "Wil Lutz", "NO", "KC"]
troySquad = ["Patrick Mahomes", "Tom Brady", 
             "Alvin Kamara", "Davante Adams", "Tyreek Hill", "Travis Kelce", 
             "Mike Evans", "Chris Godwin", "Stefon Diggs", 
             "Harrison Butker", "Ryan Succop", "KC", "TB"]
schellingSquad = ["Josh Allen", "Russell Wilson",
                  "Alvin Kamara", "Davante Adams", "Tyreek Hill", "Travis Kelce",
                  "Stefon Diggs", "DK Metcalf", "Mike Evans",
                  "Tyler Bass", "Justin Tucker", "BUF", "TB"]
jackPSquad = ["Josh Allen", "Aaron Rodgers", 
              "Davante Adams", "DK Metcalf", "Travis Kelce", "Tyreek Hill", 
              "Stefon Diggs", "Derrick Henry", "Chase Claypool",
              "Harrison Butker", "Mason Crosby", "SEA", "PIT"]
willSquad = ["Josh Allen", "Aaron Rodgers", 
             "Alvin Kamara", "John Brown", "Gabriel Davis", "Davante Adams",
             "Allen Lazard", "Tyreek Hill", "Marquise Brown",
             "Tyler Bass", "Mason Crosby", "BUF", "GB"]
steveSquad = ["Patrick Mahomes", "Aaron Rodgers", 
              "Alvin Kamara", "Davante Adams", "Aaron Jones", "Tyreek Hill",
              "Stefon Diggs", "Derrick Henry", "AJ Brown", 
              "Harrison Butker", "Tyler Bass", "KC", "BUF"]
johnRSquad = ["Patrick Mahomes", "Josh Allen", 
              "Travis Kelce", "Tyreek Hill", "Clyde Edwards-Helaire", "Aaron Jones", 
              "Antonio Brown", "Rob Gronkowski", "Chris Godwin",
              "Harrison Butker", "Ryan Succop", "LAR", "PIT"]
vinaySquad = ["Patrick Mahomes", "Russell Wilson",
              "Alvin Kamara", "Michael Thomas", "Travis Kelce", "Tyreek Hill", 
              "Stefon Diggs", "DK Metcalf", "Chris Carson", 
              "Harrison Butker", "Tyler Bass", "KC", "BUF"]
joeySquad = ["Patrick Mahomes", "Aaron Rodgers",
             "Alvin Kamara", "Michael Thomas", "Davante Adams", "Aaron Jones", 
             "Tyreek Hill", "Travis Kelce", "Stefon Diggs", 
             "Tyler Bass", "Wil Lutz", "NO", "BUF"]
spencerWileSquad = ["Patrick Mahomes", "Russell Wilson", 
                    "Davante Adams", "Aaron Jones", "DK Metcalf", "Tyler Lockett",
                    "Tyreek Hill", "Travis Kelce", "Derrick Henry", 
                    "Rodrigo Blankenship", "Ryan Succop", "SEA", "KC"]
spencerWhitSquad = ["Patrick Mahomes", "Aaron Rodgers", 
                    "Alvin Kamara", "Michael Thomas", "Travis Kelce", "Tyreek Hill", 
                    "Davante Adams", "Aaron Jones", "Stefon Diggs", 
                    "Harrison Butker", "Tyler Bass", "KC", "BUF"]
daveSquad = ["Lamar Jackson", "Tom Brady", 
             "Alvin Kamara", "Tyreek Hill", "Travis Kelce", "Aaron Jones",
             "Chris Godwin", "Mike Evans", "Stefon Diggs", 
             "Ryan Succop", "Justin Tucker", "BAL", "TB"]




In [3]:
"""CREATE TABLE THAT TELLS YOU HOW MANY ROSTERS EACH PLAYER IS ON"""

allTeams = (caseySquad + kellySquad + johnMSquad + jamesSquad + micahSquad + 
             timSquad + weezSquad + skipSquad + nickySquad + troySquad +
             schellingSquad + jackPSquad + willSquad + steveSquad + johnRSquad +
             vinaySquad + joeySquad + spencerWileSquad + spencerWhitSquad + daveSquad)  

counter = Counter(allTeams)
popular = counter.most_common()


popular_players = pd.DataFrame(popular).rename(columns = {0: "Player", 1: "Number of Teams"})
popular_players

,Player,Number of Teams
0,Tyreek Hill,19
1,Alvin Kamara,16
2,Davante Adams,14
3,Travis Kelce,14
4,Stefon Diggs,13
5,Patrick Mahomes,13
6,Harrison Butker,12
7,Tyler Bass,10
8,Aaron Jones,10
9,BUF,10


## Create Webscraping Functions

In [88]:
def get_scoring_data(input_url):
    """This function returns all the scoring plays from a given game as a list of strings"""
    """this format is for the ESPN Gamecast version of games"""
    
    assert type(input_url) == str , "input_url must be a string -- try putting it in quotes!"
    
    url = input_url
    res = requests.get(url)
    tree = html.fromstring(res.content)
    # get the info on scoring players, yardage, playtype, etc.
    scoring = tree.xpath('//div[@class="headline"]/text()')
    # get updated home and away scores
    homescore = list(map(int, tree.xpath('//td[@class="home-score"]/text()')))
    awayscore = list(map(int, tree.xpath('//td[@class="away-score"]/text()')))
    
    c = tuple(zip(awayscore, homescore))
    
    print(c)
    as_string = [str(i)for i in scoring]
    with_score = tuple(zip(as_string, c))
    return with_score

In [94]:
def scoring_by_player(team1_at_team2_string):
    """This function assigns fantasy points from a single playoff game to the respective player"""
    
    new = []
    pos = 0
    scoring_string, points_string = zip(*team1_at_team2_string)

    
    for i in scoring_string:
        if "(Two-Point Pass Conversion Failed)" in i:
            new.append((str(i.rpartition("(Two-Point Pass Conversion Failed)")[0])))
        elif "(Two-Point Run Conversion Failed)" in i:
            new.append((str(i.rpartition("(Two-Point Run Conversion Failed)")[0])))
        elif '(' in i:
            new.extend((i.split("(")))
        else:
            new.append(i)
    new = [i.strip() for i in new]
    new = [i.strip(')') for i in new]
    #new = [i.replace('to ', '') for i in new]
    #get rid of all periods in names *(ie. Jr. or A.J.)
    new = [i.replace('.', '') for i in new]
    


    newer = []
    pos = 0
    for i in new:
        if 'pass' in i:
            add = i.split('pass from')
            newer.extend(add)
            newer[pos + 1] = newer[pos + 1] + ' ' + newer[pos][-6:] + 'pass'
            newer[pos] = newer[pos] + 'pass'
            pos += 2 
        elif "Pass" in i:
            add = i.split("Pass to")
            newer.extend(add)
            newer[pos] = newer[pos] + "Two-Point Conversion"
            pos += 2
        else:
            newer.append(i)
            pos += 1
    

    newer = [i.strip() for i in newer]
    
    return newer


In [6]:
def d_p_pt(scoring_by_player_func):
    """This function splits scoring plays into 3 distinct lists of distances, players, and play_type"""
    
    import re
    import math
    #regex1 = re.compile(r"(\w*? \w*? *?) [0-9|Kick|Pass|Run|for|Two|Safety|Fumble]")
    #regex1 = re.compile(r"(\D*? \D*? *?) [0-9|Kick|Pass|Run|for|Two|Safety|Fumble]")
    #KPRTSFIf is for Kick, Pass, Run, Two, Safety, Fumble, Interception, for
    regex1 = re.compile(r"([A-Z]\D*? [A-Z]\D*? *?) [0-9KPRTSFIf]")
    regex2 = re.compile(r'\d{1,3}')
    #regex3 = re.compile(r'\w+ \w+(.*)')
    #regex3 = re.compile(r'[^to ]\w+ [\w-]+(.*)')
    regex3 = re.compile(r'[A-Z]\w+ [\w-]+[^\s](.*)')
    
    scoring = scoring_by_player_func

    player = [re.findall(regex1, str(i))[0].strip() for i in scoring]
    dist = [re.findall(regex2, str(i)) for i in scoring]
    
    play_type = [re.findall(regex3,  str(i))[0].strip().lower() for i in scoring]
    
    for i in dist:
        if i == []:
            i.append(0)
    distances = [] 
    distances = [i[0] for i in dist]
    
    return distances, player, play_type

In [7]:
def make_game_df(d_p_pt_func):

    distances = d_p_pt_func[0]
    player = d_p_pt_func[1]
    play_type = d_p_pt_func[2]

    scores = pd.DataFrame(data = {'player': player, 'distance': pd.to_numeric(distances), 'play type': play_type})

    

    scores["FG"] = 0
    scores["TD"] = 0
    scores["Safety"] = 0
    scores["PAT"] = 0
    scores["PAT2"] = 0
    scores.loc[scores["play type"].str.contains('field goal'), "FG"] = 1 
    scores.loc[scores["play type"].str.contains("kick"), "PAT"] = 1
    scores.loc[scores["play type"].str.contains("two-point"), "PAT2"] = 1
    scores.loc[scores["play type"].str.contains("safety"), "Safety"] = 1
    scores.loc[scores["play type"].str.contains("pass"), "TD"] = 1
    scores.loc[scores["play type"].str.contains("run"), "TD"] = 1
    scores.loc[scores["play type"].str.contains("return"), "TD"] = 1
    scores.loc[scores["play type"].str.contains("recovery"), "TD"] = 1
    scores.loc[scores["TD"] == scores["PAT2"], "TD"] = 0

    scores["points"] = (((scores["FG"] * scores['distance']) // 10) + 
                        ((5 * scores["TD"] + (scores["distance"]) // 10) * scores["TD"]) + 
                        (2 * scores["Safety"]) + 
                        (1 * scores["PAT"]) + 
                        (2 * scores["PAT2"]))

    scores.loc[(scores["distance"] > 55) & (scores["FG"] == 1), "points"] = scores["points"] + (scores["distance"] - 55)
    
    return scores

In [8]:
def get_single_game_points(input_url):
    """This combines all the functions written above into 1"""
    """This function's output is all the scoring (and respective points) for a single playoff game"""
    """Make sure the input_url format is for the ESPN Gamecast version of games in string form"""
    
    team1_at_team2_string = get_scoring_data(input_url)
    
    scoring_by_player_func = scoring_by_player(team1_at_team2_string)
    
    d_p_pt_func = d_p_pt(scoring_by_player_func)
    
    game_df = make_game_df(d_p_pt_func)
    
    full_game = game_df.replace({"kick": "PAT"})
    
    return full_game

##### Troubleshooting

In [33]:
e1 = get_scoring_data('https://www.espn.com/nfl/game?gameId=401220402')
#e1 = get_scoring_data('https://www.espn.com/nfl/game/_/gameId/401220394')
e1

['Mike Evans 15 Yd pass from Tom Brady (Ryan Succop Kick)', 'Marquez Valdes-Scantling 50 Yd pass from Aaron Rodgers (Mason Crosby Kick)', 'Leonard Fournette 20 Yd Run (Ryan Succop Kick)', 'Mason Crosby 24 Yd Field Goal', 'Scotty Miller 39 Yd pass from Tom Brady (Ryan Succop Kick)', 'Cameron Brate 8 Yd pass from Tom Brady (Ryan Succop Kick)', 'Robert Tonyan 8 Yd pass from Aaron Rodgers (Mason Crosby Kick)', 'Davante Adams 2 Yd pass from Aaron Rodgers (Two-Point Pass Conversion Failed)', 'Ryan Succop 46 Yd Field Goal', 'Mason Crosby 26 Yd Field Goal']


['Mike Evans 15 Yd pass from Tom Brady (Ryan Succop Kick)',
 'Marquez Valdes-Scantling 50 Yd pass from Aaron Rodgers (Mason Crosby Kick)',
 'Leonard Fournette 20 Yd Run (Ryan Succop Kick)',
 'Mason Crosby 24 Yd Field Goal',
 'Scotty Miller 39 Yd pass from Tom Brady (Ryan Succop Kick)',
 'Cameron Brate 8 Yd pass from Tom Brady (Ryan Succop Kick)',
 'Robert Tonyan 8 Yd pass from Aaron Rodgers (Mason Crosby Kick)',
 'Davante Adams 2 Yd pass from Aaron Rodgers (Two-Point Pass Conversion Failed)',
 'Ryan Succop 46 Yd Field Goal',
 'Mason Crosby 26 Yd Field Goal']

In [23]:
e1 = get_scoring_data('https://www.espn.com/nfl/game?gameId=401220402')
e2 = scoring_by_player(e1)
e2


['Mike Evans 15 Yd pass',
 'Tom Brady 15 Yd pass',
 'Ryan Succop Kick',
 'Marquez Valdes-Scantling 50 Yd pass',
 'Aaron Rodgers 50 Yd pass',
 'Mason Crosby Kick',
 'Leonard Fournette 20 Yd Run',
 'Ryan Succop Kick',
 'Mason Crosby 24 Yd Field Goal',
 'Scotty Miller 39 Yd pass',
 'Tom Brady 39 Yd pass',
 'Ryan Succop Kick',
 'Cameron Brate 8 Yd pass',
 'Tom Brady  8 Yd pass',
 'Ryan Succop Kick',
 'Robert Tonyan 8 Yd pass',
 'Aaron Rodgers  8 Yd pass',
 'Mason Crosby Kick',
 'Davante Adams 2 Yd pass',
 'Aaron Rodgers  2 Yd pass',
 'Ryan Succop 46 Yd Field Goal',
 'Mason Crosby 26 Yd Field Goal']

## Deploy Webscraping Functions

### Wild Card Round

In [9]:
"""WEBSCRAPE WILD CARD WEEKEND GAMES"""
#MAKE SURE TO INSPECT ALL THE TABLES BEFORE MOVING ON
#if needed, replace certain players names (ie Ka'imi Fairbairn or Defensive scores)
## example1: titans_at_pats = get_single_game_points('https://www.espn.com/nfl/game?gameId=401131036').replace({"Logan Ryan": "TEN"})
## example2: bills_at_texans = get_single_game_points('https://www.espn.com/nfl/game/_/gameId/401131037').replace({"imi Fairbairn": "Ka'imi Fairbairn"})

colts_at_bills = get_single_game_points('https://www.espn.com/nfl/game/_/gameId/401220393')
rams_at_seahawks = get_single_game_points('https://www.espn.com/nfl/game/_/gameId/401220372').replace({"Darious Williams": "LAR"})
bucs_at_wft = get_single_game_points('https://www.espn.com/nfl/game/_/gameId/401220371')

ravens_at_titans = get_single_game_points('https://www.espn.com/nfl/game/_/gameId/401220396')
bears_at_saints = get_single_game_points('https://www.espn.com/nfl/game/_/gameId/401220395')
browns_at_steelers = get_single_game_points('https://www.espn.com/nfl/game/_/gameId/401220394').replace({"Karl Joseph": "CLE"})


In [10]:
## INSPECT ALL TABLES
browns_at_steelers

,player,distance,play type,FG,TD,Safety,PAT,PAT2,points
0,CLE,0,fumble recovery in end zone,0,1,0,0,0,5
1,Cody Parkey,0,PAT,0,0,0,1,0,1
2,Jarvis Landry,40,40 yd pass,0,1,0,0,0,9
3,Baker Mayfield,40,40 yd pass,0,1,0,0,0,9
4,Cody Parkey,0,PAT,0,0,0,1,0,1
5,Kareem Hunt,11,11 yd run,0,1,0,0,0,6
6,Cody Parkey,0,PAT,0,0,0,1,0,1
7,Kareem Hunt,8,8 yd run,0,1,0,0,0,5
8,Cody Parkey,0,PAT,0,0,0,1,0,1
9,James Conner,1,1 yd run,0,1,0,0,0,5


In [11]:
"""CREATE WILD CARD SCORERS TABLE"""
### create table that stores all scorers from WC Weekend and how many points they scored
all_wc_scores = pd.concat([colts_at_bills, rams_at_seahawks, bucs_at_wft, ravens_at_titans, bears_at_saints, browns_at_steelers])[["player", "points"]]
wc = all_wc_scores.groupby("player").sum().reset_index()
wc_top = wc.sort_values("points", ascending = 0).reset_index().drop(columns = {"index"})
wc_top.head()

,player,points
0,Ben Roethlisberger,25
1,Baker Mayfield,23
2,Josh Allen,18
3,Russell Wilson,16
4,DK Metcalf,16


In [12]:
"""WILD CARD GAMES TABLES EXCEL EXPORTER"""
###It doesn't change the data, just puts it in an excel readable format.

writer_wc = pd.ExcelWriter("21WildCard.xlsx", engine='xlsxwriter')

colts_at_bills.to_excel(writer_wc, sheet_name = "c@b")
rams_at_seahawks.to_excel(writer_wc, sheet_name = "r@s")
bucs_at_wft.to_excel(writer_wc, sheet_name = "b@w")
ravens_at_titans.to_excel(writer_wc, sheet_name = "r@t")
bears_at_saints.to_excel(writer_wc, sheet_name = "b@s")
browns_at_steelers.to_excel(writer_wc, sheet_name ="br@s")

wc.to_excel(writer_wc, sheet_name = "all wc scoring")
wc_top.to_excel(writer_wc, sheet_name = "top wc scoring ")

### Divisional Round

In [13]:
"""WEBSCRAPE DIVISIONAL WEEKEND GAMES"""
#MAKE SURE TO INSPECT ALL THE TABLES BEFORE MOVING ON
#if needed, replace certain players names (ie Ka'imi Fairbairn or Defensive scores)
#Example1: texans_at_chiefs = get_single_game_points("https://www.espn.com/nfl/game/_/gameId/401131043").replace({"imi Fairbairn": "Ka'imi Fairbairn", "10 Yd" : "HOU", "jr. 10 yd return of blocked punt": "Blocked Punt "})
rams_at_packers = get_single_game_points('https://www.espn.com/nfl/game?gameId=401220398')
ravens_at_bills = get_single_game_points('https://www.espn.com/nfl/game/_/gameId/401220397').replace({"Taron Johnson": "BUF"})
browns_at_chiefs = get_single_game_points('https://www.espn.com/nfl/game/_/gameId/401220400')
bucs_at_saints = get_single_game_points('https://www.espn.com/nfl/game/_/gameId/401220399')


In [14]:
#INSPECT ALL TABLES
bucs_at_saints

,player,distance,play type,FG,TD,Safety,PAT,PAT2,points
0,Wil Lutz,23,23 yd field goal,1,0,0,0,0,2
1,Wil Lutz,42,42 yd field goal,1,0,0,0,0,4
2,Ryan Succop,26,26 yd field goal,1,0,0,0,0,2
3,Mike Evans,3,3 yd pass,0,1,0,0,0,5
4,Tom Brady,3,3 yd pass,0,1,0,0,0,5
5,Ryan Succop,0,PAT,0,0,0,1,0,1
6,Tre'Quan Smith,56,56 yd pass,0,1,0,0,0,10
7,Jameis Winston,56,56 yd pass,0,1,0,0,0,10
8,Wil Lutz,0,PAT,0,0,0,1,0,1
9,Ryan Succop,37,37 yd field goal,1,0,0,0,0,3


In [15]:
"""CREATE DIVISIONAL SCORERS TABLE"""
### create table that stores all scorers from Divisional Weekend and how many points they scored
all_div_scores = pd.concat([rams_at_packers, ravens_at_bills, browns_at_chiefs, bucs_at_saints])[["player", "points"]]
div = all_div_scores.groupby("player").sum().sort_values("points", ascending = False).reset_index()
div_top = div.sort_values("points", ascending = 0).reset_index().drop(columns = {"index"})
div_top

,player,points
0,Aaron Rodgers,20
1,Tre'Quan Smith,16
2,BUF,15
3,Tom Brady,15
4,Patrick Mahomes,12
5,Ryan Succop,11
6,Harrison Butker,11
7,Jameis Winston,10
8,Allen Lazard,10
9,Wil Lutz,8


In [39]:
"""DIVISIONAL GAMES EXCEL EXPORT"""
###It doesn't change the data, just puts it in an excel readable format.

writer_div = pd.ExcelWriter("21divisional.xlsx", engine='xlsxwriter')

rams_at_packers.to_excel(writer_div, sheet_name = "LAR@GB")
ravens_at_bills.to_excel(writer_div, sheet_name = "BAL@BUF")
browns_at_chiefs.to_excel(writer_div, sheet_name = "CLE@KC")
bucs_at_saints.to_excel(writer_div, sheet_name = "TB@NO")

div.to_excel(writer_div, sheet_name = "div scoring")
div_top.to_excel(writer_div, sheet_name = "top div scoring")

### Championship Round

In [16]:
"""WEBSCRAPE CHAMPIONSHIP GAMES"""
#MAKE SURE TO INSPECT ALL THE TABLES BEFORE MOVING ON
#if needed, replace certain players names (ie Ka'imi Fairbairn or Defensive scores)
bucs_at_packers = get_single_game_points('https://www.espn.com/nfl/game?gameId=401220402')
bills_at_chiefs = get_single_game_points('https://www.espn.com/nfl/game/_/gameId/401220401')


bills_at_chiefs

,player,distance,play type,FG,TD,Safety,PAT,PAT2,points
0,Tyler Bass,51,51 yd field goal,1,0,0,0,0,5
1,Dawson Knox,3,3 yd pass,0,1,0,0,0,5
2,Josh Allen,3,3 yd pass,0,1,0,0,0,5
3,Mecole Hardman,3,3 yd pass,0,1,0,0,0,5
4,Patrick Mahomes,3,3 yd pass,0,1,0,0,0,5
5,Harrison Butker,0,PAT,0,0,0,1,0,1
6,Darrel Williams,6,6 yd run,0,1,0,0,0,5
7,Harrison Butker,0,PAT,0,0,0,1,0,1
8,Clyde Edwards-Helaire,1,1 yd run,0,1,0,0,0,5
9,Harrison Butker,0,PAT,0,0,0,1,0,1


In [17]:
"""CREATE CHAMPIONSHIP SCORERS TABLE"""
### create table that stores all scorers from Divisional Weekend and how many points they scored
all_ship_scores = pd.concat([bucs_at_packers, bills_at_chiefs])[["player", "points"]]
ship = all_ship_scores.groupby("player").sum().reset_index()
ship_top = ship.sort_values("points", ascending = 0).reset_index().drop(columns = {"index"})
ship_top

,player,points
0,Aaron Rodgers,20
1,Tom Brady,19
2,Patrick Mahomes,15
3,Tyler Bass,14
4,Marquez Valdes-Scantling,10
5,Travis Kelce,10
6,Josh Allen,10
7,Harrison Butker,9
8,Scotty Miller,8
9,Ryan Succop,8


In [337]:
"""CHAMPIONSHIPS GAMES EXCEL EXPORT"""
###It doesn't change the data, just puts it in an excel readable format.



writer_ship = pd.ExcelWriter("21ship.xlsx", engine='xlsxwriter')
bucs_at_packers.to_excel(writer_ship, sheet_name = "TB@GB")
bills_at_chiefs.to_excel(writer_ship, sheet_name = "BUF@KC")

ship.to_excel(writer_ship, sheet_name = "ship scoring")
ship_top.to_excel(writer_ship, sheet_name = "ship scoring", startcol = 5)

### Super Bowl

In [19]:
"""WEBSCRAPE SUPER BOWL"""
#MAKE SURE TO INSPECT ALL THE TABLES BEFORE MOVING ON
#if needed, replace certain players names (ie Ka'imi Fairbairn or Defensive scores)
chiefs_v_bucs = get_single_game_points('https://www.espn.com/nfl/game?gameId=401220403')


chiefs_v_bucs

,player,distance,play type,FG,TD,Safety,PAT,PAT2,points
0,Harrison Butker,49,49 yd field goal,1,0,0,0,0,4
1,Rob Gronkowski,8,8 yd pass,0,1,0,0,0,5
2,Tom Brady,8,8 yd pass,0,1,0,0,0,5
3,Ryan Succop,0,PAT,0,0,0,1,0,1
4,Rob Gronkowski,17,17 yd pass,0,1,0,0,0,6
5,Tom Brady,17,17 yd pass,0,1,0,0,0,6
6,Ryan Succop,0,PAT,0,0,0,1,0,1
7,Harrison Butker,34,34 yd field goal,1,0,0,0,0,3
8,Antonio Brown,1,1 yd pass,0,1,0,0,0,5
9,Tom Brady,1,1 yd pass,0,1,0,0,0,5


In [20]:
"""SUPER BOWL GAME EXCEL EXPORT"""
###Same as above...This won't run from Google Colab (Written using Jupyter Notebook, but copied here for sharing purposes)
###It doesn't change the data, just puts it in an excel readable format.

writer_sb = pd.ExcelWriter("sb21.xlsx", engine='xlsxwriter')
chiefs_v_bucs.to_excel(writer_sb, sheet_name = "Super Bowl")

all_sb_scores = pd.concat([chiefs_v_bucs])[["player", "points"]]
sb = all_sb_scores.groupby("player").sum().sort_values("points", ascending = False).reset_index()
sb.to_excel(writer_sb, sheet_name = "Super Bowl scoring")

## Finalize Teams & Assign Points

In [24]:
def make_team(squad, name):
    """CREATE DF WITH EACH PLAYER ON A (FANTASY) ROSTER AND THEIR POINT TOTALS, BY ROUND"""
    team = pd.DataFrame(squad)
    
    """Adding Wild Card results"""
    team = team.merge(wc, how = "left", left_on = 0, right_on = "player")
    team = team.drop(columns = "player").rename(columns = {0: "player"}).fillna(0)
    
    """Adding Divisional Round results"""
    team = team.merge(div, how = "left", left_on = "player" , right_on = "player").fillna(0)
    team = team.rename(columns = {"points_x": "WC", "points_y": "Div"})
    
    """Adding Conference Championship results"""
    team = team.merge(ship, how = 'left', left_on = "player", right_on = "player").fillna(0)
    team = team.rename(columns = {"points": "Ship"})
    
    """Adding Super Bowl results"""
    team = team.merge(sb, how = 'left', left_on = "player", right_on = "player").fillna(0)
    team = team.rename(columns = {"points": "SB"})
    
    """Creating a Total Column that sums all the other columns"""
    team["Total"] = team["WC"] + team["Div"] + team["Ship"] + team["SB"]
    team = team.append(team.sum().rename('Total'))
    # clearing long list of player names for format purpose
    team.at['Total', 'player']  = name
    # extracting total points
    total = team.at["Total", "Total"]
    
    team_point_dict[name] = total
    
    
    return team

In [25]:
"""COMBINING PLAYERS' SCORING DATA AND PARTICIPANTS' TEAMS"""

# create a dictionary that stores {team_name : total_points}
team_point_dict = {}

casey = make_team(caseySquad, "casey") 
kelly = make_team(kellySquad, "kelly") 
johnM = make_team(johnMSquad, "johnM")
james = make_team(jamesSquad, "james") 
micah = make_team(micahSquad, "micah") 
tim = make_team(timSquad, "tim") 
ben = make_team(weezSquad, "ben") 
skip = make_team(skipSquad, "skip") 
nicky = make_team(nickySquad, "nicky") 
troy = make_team(troySquad, "troy") 
schelling = make_team(schellingSquad, "schelling") 
jackP = make_team(jackPSquad, "jackP") 
will = make_team(willSquad, "will") 
steve = make_team(steveSquad, "steve") 
johnR = make_team(johnRSquad, "johnR") 
vinay = make_team(vinaySquad, "vinay") 
joey = make_team(joeySquad, "joey") 
spencerWile = make_team(spencerWileSquad, "spencerWile") 
spencerWhit = make_team(spencerWhitSquad, "spencerWhit") 
dave = make_team(daveSquad, "dave")



In [26]:
"""ORDER TEAMS BY TOTAL POINTS"""

## using the team_point_dict dictionary, sort players names by total points, 
## then create a new dict
sorted_tuples = sorted(team_point_dict.items(), key=lambda item: item[1], reverse = True)
team_by_points = {k: v for k, v in sorted_tuples}

## inspect team_by_points to make sure they are, in fact, in order
team_by_points


{'troy': 226.0,
 'johnM': 224.0,
 'casey': 214.0,
 'spencerWhit': 200.0,
 'johnR': 198.0,
 'kelly': 196.0,
 'tim': 190.0,
 'jackP': 186.0,
 'dave': 184.0,
 'steve': 183.0,
 'schelling': 179.0,
 'joey': 179.0,
 'vinay': 177.0,
 'skip': 174.0,
 'micah': 173.0,
 'ben': 160.0,
 'nicky': 160.0,
 'james': 158.0,
 'will': 156.0,
 'spencerWile': 134.0}

In [121]:
sb

,player,points
0,Tom Brady,16
1,Harrison Butker,12
2,Rob Gronkowski,11
3,Ryan Succop,9
4,Leonard Fournette,7
5,Antonio Brown,5


In [127]:
first2 = pd.concat([wc_top, div_top]).groupby(['player']).sum().reset_index().sort_values("points", ascending = False)
conf = pd.concat([first2, ship_top]).groupby(['player']).sum().reset_index().sort_values("points", ascending = False).reset_index(drop = "True")
final_all_scorers = pd.concat([conf, sb]).groupby(['player']).sum().reset_index().sort_values("points", ascending = False).reset_index(drop = "True")
final_all_scorers

,player,points
0,Tom Brady,65
1,Aaron Rodgers,40
2,Ryan Succop,39
3,Josh Allen,33
4,Harrison Butker,32
...,...,...
70,JuJu Smith-Schuster,5
71,Latavius Murray,5
72,Zach Pascal,5
73,Rodrigo Blankenship,4


## Export Fantasy Teams & Points to Excel

In [28]:
"""PUT FANTASY TEAMS & POINTS INTO EXCEL FORMAT"""

##This cell doesn't change the data, just puts it in an excel readable format.
##iterate through team_by_points dict, then puts corresponding df in the leaderboard worksheet

#CHANGE NEXT LINE EACH WEEK TO EXPORT TO A NEW EXCEL SPREADSHEET
writer = writer_sb

count = 0
for name in team_by_points.keys():
    row = (count % 2 != 0) * 16
    col = (count // 2) * 7
    df = locals()[name]
    df.to_excel(writer, sheet_name="leaderboard", startrow=row , startcol = col)
    count += 1
    


In [29]:
"""PUT LIST OF PLAYERS BY # OF ROSTERS INTO EXCEL FORMAT"""
## This df is created at the very start of the notebook
popular_players.to_excel(writer, sheet_name = "popular")

In [30]:
"""EXPORT EVERYTHING TO EXCEL"""
writer.save()

## Other

In [128]:
writer_whoops = pd.ExcelWriter("21SBWhoops.xlsx", engine='xlsxwriter')
final_all_scorers.to_excel(writer_whoops, sheet_name = "final all scorers")
writer_whoops.save()